# Predict low high models
Model selection for predict lo hi


To install conda and binance packages to this notebook uncomment the code below

In [1]:
#%conda install -c plotly plotly=5.9.0
#%conda install pip
#%conda install twisted
%pip install plotly==5.9.0
%pip install twisted
#%pip install binance-connector==1.13.0
%pip install pandas
%pip install scikit-learn
%pip install tensorflow==2.11.0
%pip install keras==2.11.0
%pip install scikeras


import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

Keyring is skipped due to an exception: 'keyring.backends'
Note: you may need to restart the kernel to use updated packages.
Keyring is skipped due to an exception: 'keyring.backends'
Note: you may need to restart the kernel to use updated packages.
Keyring is skipped due to an exception: 'keyring.backends'
Note: you may need to restart the kernel to use updated packages.
Keyring is skipped due to an exception: 'keyring.backends'
Note: you may need to restart the kernel to use updated packages.
Keyring is skipped due to an exception: 'keyring.backends'


Note: you may need to restart the kernel to use updated packages.
Keyring is skipped due to an exception: 'keyring.backends'
Note: you may need to restart the kernel to use updated packages.
Keyring is skipped due to an exception: 'keyring.backends'
Note: you may need to restart the kernel to use updated packages.


## Feature engineering
Read data from bot's history, resample to equal intervals, create X, y

In [2]:
import os
import glob
import pandas as pd
from datetime import timedelta,date

def read_last_data(strategy):
    """ Read last last day from bot history """
    
    def read_last(symbol: str, name: str):
        data_dir=f"./../data/{strategy}/Xy"
        file_path = sorted([f for f in os.listdir(data_dir) if f.endswith(f"{symbol}_{name}.csv")])[-1]
        return pd.read_csv(f"{data_dir}/{file_path}", parse_dates=True, index_col="datetime")

    data=read_last("BTCUSDT", "data")
    X=read_last("BTCUSDT", "X")
    y=read_last("BTCUSDT", "y")
    return data,X,y

strategy="SimpleKerasStrategy"
bidask,X_bot,y_bot = read_last_data(strategy)
bidask

,symbol,bid,bid_vol,ask,ask_vol
datetime,,,,,
2023-04-25 23:59:45.897283,BTCUSDT,28343.64,2.72995,28343.65,0.15772
2023-04-25 23:59:46.413752,BTCUSDT,28343.64,2.74760,28343.65,0.15772
2023-04-25 23:59:46.771466,BTCUSDT,28343.64,2.76525,28343.65,0.15772
2023-04-25 23:59:47.119257,BTCUSDT,28343.64,2.74760,28343.65,0.15772
2023-04-25 23:59:47.409589,BTCUSDT,28343.64,2.74760,28343.65,0.14810
...,...,...,...,...,...
2023-04-26 20:11:33.060399,BTCUSDT,28192.74,1.57890,28192.75,0.15587
2023-04-26 20:11:33.235846,BTCUSDT,28192.74,0.37890,28192.75,0.15587
2023-04-26 20:11:33.409391,BTCUSDT,28192.74,0.37890,28194.00,0.00657


In [3]:

def get_targets(bidask):
        """ 
        Calculate targets - bid/ask bounds in future prediction window
        """
        predict_window="60s"
        fut_min=bidask[["bid", "ask"]][::-1].rolling(predict_window).min()[::-1].rename(columns={"bid":"bid_min_fut", "ask":"ask_min_fut"})
        fut_max=bidask[["bid", "ask"]][::-1].rolling(predict_window).max()[::-1].rename(columns={"bid":"bid_max_fut", "ask":"ask_max_fut"})
        return pd.concat([fut_min, fut_max], axis = 1)

y_raw = get_targets(bidask)
pd.concat([X_bot,y_raw], axis=1).dropna()

,time_hour,time_minute,time_second,time_day_of_week,time_diff,spread,bid_diff,bid_vol_diff,ask_diff,ask_vol_diff,l2_bucket_-10,l2_bucket_-9,l2_bucket_-8,l2_bucket_-7,l2_bucket_-6,l2_bucket_-5,l2_bucket_-4,l2_bucket_-3,l2_bucket_-2,l2_bucket_-1,l2_bucket_0,l2_bucket_1,l2_bucket_2,l2_bucket_3,l2_bucket_4,l2_bucket_5,l2_bucket_6,l2_bucket_7,l2_bucket_8,l2_bucket_9,bid_min_fut,ask_min_fut,bid_max_fut,ask_max_fut
datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-04-25 23:59:45.897283,23,59,45,1,466647000.0,0.01,0.0,0.01765,0.00,0.00000,0.00086,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0000,35.01457,32.53753,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,28159.58,28160.68,29999.99,30000.00
2023-04-25 23:59:46.413752,23,59,46,1,516469000.0,0.01,0.0,0.01765,0.00,0.00000,0.00086,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0000,35.01457,32.53753,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,28159.58,28160.68,29999.99,30000.00
2023-04-25 23:59:46.771466,23,59,46,1,357714000.0,0.01,0.0,0.01765,0.00,0.00000,0.00086,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0000,35.01457,32.53753,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,28159.58,28160.68,29999.99,30000.00
2023-04-25 23:59:47.119257,23,59,47,1,347791000.0,0.01,0.0,-0.01765,0.00,0.00000,0.00086,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0000,35.01457,32.53753,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,28159.58,28160.68,29999.99,30000.00
2023-04-25 23:59:47.409589,23,59,47,1,290332000.0,0.01,0.0,0.00000,0.00,-0.00962,0.00086,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0000,35.01457,32.53753,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,28159.58,28160.68,29999.99,30000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-26 20:11:33.060399,20,11,33,2,174520000.0,0.01,0.0,-0.80000,0.00,0.00000,0.00080,0.0,0.0,0.0,159.31959,0.0,0.0,0.0,0.2372,8.67194,7.64158,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00028,28192.74,28192.75,28192.74,28194.00
2023-04-26 20:11:33.235846,20,11,33,2,175447000.0,0.01,0.0,-1.20000,0.00,0.00000,0.00080,0.0,0.0,0.0,159.31959,0.0,0.0,0.0,0.2372,8.67194,7.64158,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00028,28192.74,28192.75,28192.74,28194.00
2023-04-26 20:11:33.409391,20,11,33,2,173545000.0,1.26,0.0,0.00000,1.25,-0.14930,0.00080,0.0,0.0,0.0,159.31959,0.0,0.0,0.0,0.2372,8.67194,7.64158,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00028,28192.74,28193.99,28192.74,28194.00


Resample data to use equal time intervals

In [4]:
def resampled(X: pd.DataFrame, y: pd.DataFrame) -> (pd.DataFrame, pd.DataFrame):
    """ Resample to make equal intervals time series """
    time_interval="10s"
    Xy=pd.concat([X,y], axis=1)
    diffmap = [(c, "sum") for c in Xy.columns if c.endswith("diff")]
    timemap = [(c, "last") for c in Xy.columns if c.startswith("time") and not c.endswith("diff")]
    l2map = [(c, "last") for c in Xy.columns if c.startswith("l2_")]
    
    futmap = [(c, "last") for c in Xy.columns if c.endswith("_fut")]
    colmap = dict(diffmap + timemap + l2map + futmap + [("spread", "last")])
    resampled = Xy.resample(time_interval).agg(colmap).dropna()
    return resampled[X.columns], resampled[y.columns]
    
#X,y = resampled(X_bot, y_raw)
X,y=X_bot,y_raw
pd.concat([X,y], axis=1)

,time_hour,time_minute,time_second,time_day_of_week,time_diff,spread,bid_diff,bid_vol_diff,ask_diff,ask_vol_diff,l2_bucket_-10,l2_bucket_-9,l2_bucket_-8,l2_bucket_-7,l2_bucket_-6,l2_bucket_-5,l2_bucket_-4,l2_bucket_-3,l2_bucket_-2,l2_bucket_-1,l2_bucket_0,l2_bucket_1,l2_bucket_2,l2_bucket_3,l2_bucket_4,l2_bucket_5,l2_bucket_6,l2_bucket_7,l2_bucket_8,l2_bucket_9,bid_min_fut,ask_min_fut,bid_max_fut,ask_max_fut
datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-04-25 23:59:45.897283,23,59,45,1,466647000.0,0.01,0.0,0.01765,0.00,0.00000,0.00086,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0000,35.01457,32.53753,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,28159.58,28160.68,29999.99,30000.00
2023-04-25 23:59:46.413752,23,59,46,1,516469000.0,0.01,0.0,0.01765,0.00,0.00000,0.00086,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0000,35.01457,32.53753,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,28159.58,28160.68,29999.99,30000.00
2023-04-25 23:59:46.771466,23,59,46,1,357714000.0,0.01,0.0,0.01765,0.00,0.00000,0.00086,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0000,35.01457,32.53753,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,28159.58,28160.68,29999.99,30000.00
2023-04-25 23:59:47.119257,23,59,47,1,347791000.0,0.01,0.0,-0.01765,0.00,0.00000,0.00086,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0000,35.01457,32.53753,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,28159.58,28160.68,29999.99,30000.00
2023-04-25 23:59:47.409589,23,59,47,1,290332000.0,0.01,0.0,0.00000,0.00,-0.00962,0.00086,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0000,35.01457,32.53753,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,28159.58,28160.68,29999.99,30000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-26 20:11:33.060399,20,11,33,2,174520000.0,0.01,0.0,-0.80000,0.00,0.00000,0.00080,0.0,0.0,0.0,159.31959,0.0,0.0,0.0,0.2372,8.67194,7.64158,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00028,28192.74,28192.75,28192.74,28194.00
2023-04-26 20:11:33.235846,20,11,33,2,175447000.0,0.01,0.0,-1.20000,0.00,0.00000,0.00080,0.0,0.0,0.0,159.31959,0.0,0.0,0.0,0.2372,8.67194,7.64158,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00028,28192.74,28192.75,28192.74,28194.00
2023-04-26 20:11:33.409391,20,11,33,2,173545000.0,1.26,0.0,0.00000,1.25,-0.14930,0.00080,0.0,0.0,0.0,159.31959,0.0,0.0,0.0,0.2372,8.67194,7.64158,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00028,28192.74,28193.99,28192.74,28194.00


## Create the model


In [5]:
from keras import Sequential, Input
from keras.layers import Dense, Dropout
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from scikeras.wrappers import KerasRegressor
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.compose import ColumnTransformer, make_column_selector


test_size=0.3
test_index = int(len(X)*(1-test_size))
X_train, y_train, X_test, y_test = X.iloc[:test_index], y.iloc[:test_index], X.iloc[test_index:], y.iloc[test_index:]

   
def create_model(X_train, y_train, window_size):
        
        model = Sequential()
        model.add(LSTM(128,  return_sequences=True, input_shape=(window_size, X_train.shape[1])))
        model.add(Dropout(0.2))
        model.add(LSTM(32))         
        model.add(Dropout(0.2))
        model.add(Dense(20, activation='relu'))
        model.add(Dense(y_train.shape[1], activation='linear'))
        #model.add(Dense(y_train.shape[1], activation='softmax'))
        model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mean_squared_error'])
        return model
    
    
x_scaler, y_scaler = StandardScaler().fit(X_train), StandardScaler().fit(y_train)
X_train = x_scaler.transform(X_train)
y_train = y_scaler.transform(y_train)

window_size = 10
gen = TimeseriesGenerator(X_train, y_train, length=window_size,
                               sampling_rate=1, batch_size=1)

model = create_model(X_train, y_train, window_size)
model.fit(gen,steps_per_epoch=1000, epochs=10)

Epoch 1/10
1000/1000 [==============================] - 11s 8ms/step - loss: 0.1816 - mean_squared_error: 0.2942
Epoch 2/10
1000/1000 [==============================] - 8s 8ms/step - loss: 0.0996 - mean_squared_error: 0.1511
Epoch 3/10
1000/1000 [==============================] - 8s 7ms/step - loss: 0.1159 - mean_squared_error: 0.2160
Epoch 4/10
1000/1000 [==============================] - 8s 8ms/step - loss: 0.0611 - mean_squared_error: 0.0913
Epoch 5/10
1000/1000 [==============================] - 8s 8ms/step - loss: 0.0707 - mean_squared_error: 0.1124
Epoch 6/10
1000/1000 [==============================] - 8s 8ms/step - loss: 0.0629 - mean_squared_error: 0.0874
Epoch 7/10
1000/1000 [==============================] - 8s 8ms/step - loss: 0.0682 - mean_squared_error: 0.0957
Epoch 8/10
1000/1000 [==============================] - 8s 8ms/step - loss: 0.1087 - mean_squared_error: 0.2247
Epoch 9/10
1000/1000 [==============================] - 9s 8ms/step - loss: 0.0574 - mean_squared_error

In [6]:
########### Refac attempt

from keras import Sequential, Input
from keras.layers import Dense, Dropout
from sklearn.base import BaseEstimator,TransformerMixin
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, FunctionTransformer
from scikeras.wrappers import KerasRegressor
from keras.layers import Dense,LSTM, Reshape
from sklearn.compose import ColumnTransformer, make_column_selector


test_size=0.3
test_index = int(len(X)*(1-test_size))
X_train, y_train, X_test, y_test = X.iloc[:test_index], y.iloc[:test_index], X.iloc[test_index:], y.iloc[test_index:]

   
def create_model(X_size, y_size, window_size):
        
        model = Sequential()
        model.add(LSTM(128,  return_sequences=True, input_shape=(window_size, X_size)))
        model.add(Dropout(0.2))
        model.add(LSTM(32))         
        model.add(Dropout(0.2))
        model.add(Dense(20, activation='relu'))
        model.add(Dense(y_size, activation='linear'))
        #model.add(Dense(y_train.shape[1], activation='softmax'))
        model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mean_squared_error'])
        return model
    
    
def create_pipe(X: pd.DataFrame, y: pd.DataFrame) -> TransformedTargetRegressor:

    def reshape(data):
        return data.reshape(data.shape[0], window_size, data.shape[1])

    model=create_model(X_size=X.values.shape[1], y_size=y.values.shape[1], window_size = window_size)
    regressor = KerasRegressor(model=model, epochs=10, batch_size=window_size*100, verbose=1)
    
    xpipe = Pipeline([('xscaler', StandardScaler()), 
                     ('reshape', FunctionTransformer(reshape, validate=False)),
                     ('model', regressor)])
    ypipe = Pipeline([
            ('yscaler', StandardScaler())
            #('reshape', FunctionTransformer(reshape, validate=False))
        ])
    
    # Add y transformer
    wrapped = TransformedTargetRegressor(regressor=xpipe, transformer=ypipe)
    return wrapped
    
window_size = 1

# For learning
# gen = TimeseriesGenerator(X_train, y_train, length=window_size,
#                                sampling_rate=1, batch_size=1)
#model = create_model(X_train, y_train, window_size)
#model.fit(gen,y_train)


model = create_pipe(X_train, y_train)
f=model.fit(X_train, y_train)
print("")

INFO:tensorflow:Assets written to: ram://c252101703df4017a841999ff1b3669e/assets


INFO:tensorflow:Assets written to: ram://c252101703df4017a841999ff1b3669e/assets


Epoch 1/10
1805/1805 [==============================] - 10s 5ms/step - loss: 0.0801 - mean_squared_error: 0.1297
Epoch 2/10
1805/1805 [==============================] - 9s 5ms/step - loss: 0.0239 - mean_squared_error: 0.0112
Epoch 3/10
1805/1805 [==============================] - 9s 5ms/step - loss: 0.0196 - mean_squared_error: 0.0078
Epoch 4/10
1805/1805 [==============================] - 9s 5ms/step - loss: 0.0175 - mean_squared_error: 0.0064
Epoch 5/10
1805/1805 [==============================] - 9s 5ms/step - loss: 0.0163 - mean_squared_error: 0.0057
Epoch 6/10
1805/1805 [==============================] - 9s 5ms/step - loss: 0.0152 - mean_squared_error: 0.0050
Epoch 7/10
1805/1805 [==============================] - 9s 5ms/step - loss: 0.0141 - mean_squared_error: 0.0044
Epoch 8/10
1805/1805 [==============================] - 9s 5ms/step - loss: 0.0119 - mean_squared_error: 0.0034
Epoch 9/10
1805/1805 [==============================] - 9s 5ms/step - loss: 0.0094 - mean_squared_error